<html>
     <body>
         <p><font size="6" color="white">Sign Language MNIST with CNN in Tensorflow 2.0 & Image Aumentation.</font></p>
     </body>     ![tensorflow](https://www.gstatic.com/devrel-devsite/v0bcab1972c3b1579b82e221c810ad3eb061cbd75c8520328bda82087ceb07528/tensorflow/images/lockup.svg)

The idea of this notebook it to predict Sign language competion using TF 2.0 (keras) and Data augmentation

In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

print(tf.__version__)
print(os.listdir('../input/'))

##  Define the Directory and Files

In [ ]:
name_file_train = 'sign_mnist_train.csv'
name_file_test ='sign_mnist_test.csv'
directory = '../input/sign-language-mnist/'

file_train = os.path.join(directory,name_file_train)
file_test = os.path.join(directory, name_file_test)

print(directory)
print(file_train)
print(file_test)

### Get_data function
 It is function that recieve the filename path and **return two objets**.
 1.  training images
 2.  label images
 
The task it was to created:
 1. a dataframe from file using pandas library
 2. isolate the column label.
 3. Get the array images
 4. Transform the list on np.array and gives them it a 28 to 28 shape for each line


In [ ]:
def get_data(filename):
    df = pd.read_csv(filename, header=0)
    labels= np.array(df.iloc[:,0].values)
    imgs = df.iloc[:,1:].values
    data = []
    for img in imgs:
        tmp =np.array( np.array_split(img,28))
        data.append(tmp)
    
    data = np.array(data).astype('float')    
    return data, labels

training_images, training_labels = get_data(file_train)
testing_images, testing_labels = get_data(file_test)

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

### Expand Dimension

 We define the Matrix on Training and Label. We need to include the color channel (a new dimension) and the because is black and white the value of the channel it is one.

In [ ]:
training_images = np.expand_dims(training_images,axis=3)
testing_images = np.expand_dims(testing_images,axis=3)

print(training_images.shape)
print(testing_images.shape)

### Image Generation

 We use the ImageDataGenerator, this class not override the information of the folder. Only expand the information on memory. 
 We rotate, shift and zoom. And we fill the picture on the transformation with the neares picture.

In [ ]:
# Create an ImageDataGenerator 
# and do Image Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2, 
    horizontal_flip=True,
    fill_mode='nearest'
    )

validation_datagen = ImageDataGenerator(
    rescale=1.0/255.0)

### Sequential Model
 We create 3 convolutional layers each one´s with their on MaxPooling. Before to create the Dense Layer we Dropout a twenty percent to avoid overfitting

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')    
])

model.summary()

### Model Compilation and Monitoring variables
 The compilation that we use is 'adam' or we can consider other that work we momentun. (not GD)

In [ ]:
# Compile Model. 
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit_generator(
    train_datagen.flow(training_images,training_labels,batch_size=32),
    steps_per_epoch=len(training_images)/32,
    epochs=50,
    validation_data=validation_datagen.flow(testing_images, testing_labels, batch_size=32),
    validation_steps=len(testing_images)/32
                             )

model.evaluate(testing_images, testing_labels)

### visualization
 We visualize the training and validation history

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()